In [1]:
import numpy as np
import copy
import pandas as pd
from numpy.linalg import solve, norm,cond
from scipy.linalg import hilbert

In [2]:
def qr(a): #алгоритм QR-разложения методом вращений
    n = a.shape[0]
    q, r = np.identity(n), copy.copy(a)
    for i in range(n):
        for j in range(i+1,n):
            c = r[i,i]/(r[i,i]**2 + (r[j,i]**2))**0.5
            s = r[j,i]/(r[i,i]**2 + (r[j,i]**2))**0.5
            r[i,:], r[j,:] = c*r[i,:] + s*r[j,:], -s*r[i,:] + c*r[j,:]
            q[:,i], q[:,j] = c*q[:,i] + s*q[:,j], -s*q[:,i] + c*q[:,j]
    return q,r

In [3]:
def qr_solve(q,r,b=None): #решение СЛАУ QR-методом
    if b is None:
        b = np.random.uniform(-100,100,size=(q.shape[0]))
    n = r.shape[1]
    x, y = np.zeros(n), np.dot(np.transpose(q),b)
    for j in range(len(y)):
        x[n-j-1]=(y[n-j-1]-sum([r[n-j-1,n-p-1]*x[n-p-1] for p in range(j)]))/r[n-j-1,n-j-1]
    return x

Проверим, что решения, полученные qr-методом, совпадают с решением системы:

In [25]:
a = np.random.rand(2,2)
b = np.random.rand(2)
q,r = qr(a)
norm(qr_solve(q,r,b)-solve(a,b))

5.551115123125783e-16

In [6]:
matrixes = [hilbert(n) for n in range(3,6)]

In [26]:
def regularisation_solution(a,b=None):
    if b is None:
        b = np.random.uniform(-100,100,size=(a.shape[1]))
    ans = pd.DataFrame(columns=["alpha","cond(a+alpha*E)","||x-x_alpha||","||b-A*x_alpha||"])
    q,r = qr(a)
    x = qr_solve(q,r,b)
    ans = ans.append(pd.Series([0,cond(a),norm(x-solve(a,b)),norm(b-a@x)],index=ans.columns),True)
    E = np.identity(a.shape[0])
    x = solve(a,b)
    for i in range(2,13,2):
        a_i = a + 10**(-i)*E
        q,r = qr(a_i)
        x_i = qr_solve(q,r,b)
        ans = ans.append(pd.Series([10**(-i),cond(a_i),norm(x_i-x),norm(b-a@x_i)],index=ans.columns),True)
    return ans

In [37]:
regularisation_solution(matrixes[0],np.array([1,1,1])) #результат для матрицы Гильберта 3-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,524.056778,3.288951e-13,2.512148e-15
1,1.000000e-02,111.790091,3.009003e+01,9.450946e-02
2,1.000000e-04,505.291334,1.369522e+00,3.717896e-03
3,1.000000e-06,523.862213,1.419955e-02,3.852163e-05
4,1.000000e-08,524.054831,1.420478e-04,3.853556e-07
5,1.000000e-10,524.056758,1.420484e-06,3.853571e-09
6,1.000000e-12,524.056777,1.420472e-08,3.853497e-11


In [14]:
regularisation_solution(matrixes[1],np.array([1,1,1,1])) #результат для матрицы Гильберта 4-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,15513.738739,6.930983e-12,1.123467e-14
1,1.000000e-02,149.575003,2.326094e+02,1.171839e-01
2,1.000000e-04,7627.334553,1.191303e+02,1.181379e-02
3,1.000000e-06,15354.963172,2.398420e+00,2.334470e-04
4,1.000000e-08,15512.134730,2.422972e-02,2.358064e-06
5,1.000000e-10,15513.722697,2.423221e-04,2.358303e-08
6,1.000000e-12,15513.738579,2.423265e-06,2.358332e-10


In [36]:
regularisation_solution(hilbert(5),hilbert(5)@np.ones(5)) #результат для матрицы Гильберта 5-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,476607.250242,2.165921e-11,6.843874e-16
1,1.000000e-02,157.653234,1.300612e-01,2.195547e-02
2,1.000000e-04,15172.641273,1.167345e-02,2.235339e-04
3,1.000000e-06,365456.558250,1.104924e-03,2.236058e-06
4,1.000000e-08,475162.081827,1.431874e-05,2.236068e-08
5,1.000000e-10,476592.755044,1.436502e-07,2.236066e-10
6,1.000000e-12,476607.105284,1.354233e-09,2.236118e-12
